Для выполнения домашнего задания необходимо взять файл boston house-prices datase (sklearn.datasets.load_boston) и построить модель.
По аналогии задания к лекции «Классификация: Логистическая регрессия и SVM».
А именно построить модель регрессии, дерево решений, оптимизировать параметры и сравнить результаты.

Модель регресси была построена в предыдущем домашнем задании:
https://github.com/xbbt/Netology_DS_course/blob/master/ML_intro/data_omissions/Boston_house_price_model_tuning.ipynb

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler

parameters = {
    'axes.labelsize':  14,
    'figure.titlesize': 16,
    'xtick.labelsize': 12,
    'ytick.labelsize': 12,
    'axes.titlesize': 16,
    'legend.fontsize': 12,
    'legend.title_fontsize': 14
}
plt.rcParams.update(parameters)

Читаем данные

In [2]:
from sklearn.datasets import load_boston
import pandas as pd
data = pd.read_csv(load_boston()['filename'], skiprows=1)

In [3]:
data.head()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,MEDV
0,0.00632,18.0,2.31,0,0.538,6.575,65.2,4.0900,1,296,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0,0.469,6.421,78.9,4.9671,2,242,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0,0.469,7.185,61.1,4.9671,2,242,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0,0.458,6.998,45.8,6.0622,3,222,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0,0.458,7.147,54.2,6.0622,3,222,18.7,396.90,5.33,36.2


In [4]:
print(load_boston()['DESCR'])

.. _boston_dataset:

Boston house prices dataset
---------------------------

**Data Set Characteristics:**  

    :Number of Instances: 506 

    :Number of Attributes: 13 numeric/categorical predictive. Median Value (attribute 14) is usually the target.

    :Attribute Information (in order):
        - CRIM     per capita crime rate by town
        - ZN       proportion of residential land zoned for lots over 25,000 sq.ft.
        - INDUS    proportion of non-retail business acres per town
        - CHAS     Charles River dummy variable (= 1 if tract bounds river; 0 otherwise)
        - NOX      nitric oxides concentration (parts per 10 million)
        - RM       average number of rooms per dwelling
        - AGE      proportion of owner-occupied units built prior to 1940
        - DIS      weighted distances to five Boston employment centres
        - RAD      index of accessibility to radial highways
        - TAX      full-value property-tax rate per $10,000
        - PTRATIO  pu

In [5]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error

Разбиваем датасет на train, test, valid подвыборки с тем же random_state, что и в случае регрессии. Мы собираемся подобрать оптимальные параметры деревьев методом кросс-валидации, поэтому объединяем train и test, а результат будем проверять на valid выборке.

In [6]:
data_columns_list = list(data.columns)
target_position = data_columns_list.index('MEDV')
del data_columns_list[target_position]
X = data[data_columns_list]
y = data['MEDV']

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=18)
X_test, X_valid, y_test, y_valid = train_test_split(X_test, y_test, random_state=18)

In [7]:
X_train = X_train.append(X_test, ignore_index=True)
y_train = y_train.append(y_test, ignore_index=True)

In [8]:
X_train.shape

(474, 13)

In [9]:
regressor = DecisionTreeRegressor(random_state=0)
regressor.fit(X_train, y_train)

DecisionTreeRegressor(random_state=0)

In [10]:
mean_absolute_error(regressor.predict(X_valid), y_valid)

2.0624999999999996

Без подобра гиперпараметров модели результат немного лучше, чем первоначальный результат линейной модели.

С помощью gridsearch подберём оптимальные параметры модели.

In [11]:
from sklearn.model_selection import GridSearchCV

In [12]:
parameters = {
    'criterion':('mse', 'mae', 'friedman_mse', 'poisson'),
    'splitter':('best', 'random'),
    'max_depth':[2, 3, 4, 5, 6],
    'min_samples_split':[2, 3, 4, 5, 7, 8, 9, 10],
    'min_samples_leaf': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
    'ccp_alpha':[0, 0.01, 0.1, 0.3, 0.5]
}
regressor = DecisionTreeRegressor(random_state=0)
clf = GridSearchCV(regressor, parameters, scoring='neg_mean_absolute_percentage_error')
clf.fit(X_train, y_train)

GridSearchCV(estimator=DecisionTreeRegressor(random_state=0),
             param_grid={'ccp_alpha': [0, 0.01, 0.1, 0.3, 0.5],
                         'criterion': ('mse', 'mae', 'friedman_mse', 'poisson'),
                         'max_depth': [2, 3, 4, 5, 6],
                         'min_samples_leaf': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
                         'min_samples_split': [2, 3, 4, 5, 7, 8, 9, 10],
                         'splitter': ('best', 'random')},
             scoring='neg_mean_absolute_percentage_error')

In [13]:
mean_absolute_error(clf.predict(X_valid), y_valid)

2.435415379881994

После gridsearch результат получился немного хуже, чем от применения дерева в лоб. Видимо модель становится переобученной на treain выборке.

В целом, результыт получаются похожи на результаты линейной регрессии.